In [1]:
import requests
import pandas as pd
import geopandas as gpd
import json
from wilcoxon import sheets
from wilcoxon.utils import *
import itertools
import shapely
import contextily as ctx

In [2]:
bus = sheets.getGeo("Bus")
malls = sheets.getGeo("Malls Alternative")

# Bus Routes & Stops

In [3]:
busRoutes = []
starting = 0
while True:
#     print(starting)
    try:
        headers = {'AccountKey':'XaA8uKDzTjevY+gR1sjz0A=='}
        results = requests.get('http://datamall2.mytransport.sg/ltaodataservice/BusRoutes?$skip=' + str(starting),headers=headers).text
        results = json.loads(results)
        if len(results['value']) == 0:
            break
        busRoutes.extend(results['value'])
        starting += 500
    except: break
        
busStops = []
starting = 0
while True:
#     print(starting)
    try:
        headers = {'AccountKey':'XaA8uKDzTjevY+gR1sjz0A=='}
        results = requests.get('http://datamall2.mytransport.sg/ltaodataservice/BusStops?$skip=' + str(starting),headers=headers).text
        results = json.loads(results)
        if len(results['value']) == 0:
            break
        busStops.extend(results['value'])
        starting += 500
    except: break
        
pd.DataFrame(busRoutes).merge(pd.DataFrame(busStops)).to_clipboard(index=False)

In [162]:
serviceses = []
for lat1, long1 in malls[["Latitude","Longitude"]].values.tolist():
    services = []
    for lat2, long2, service in bus[["Latitude","Longitude","ServiceNo"]].values.tolist():
        distance = dist(lat1, long1, lat2, long2)
        if distance < 0.5:
            services.append(service)
        
    services = list(set(services))
    serviceses.append([len(services), ", ".join(services)])
    
pd.DataFrame(serviceses).to_clipboard(index=False, header=False)

# Parking Lots

In [165]:
parking = []
starting = 0
while True:
#     print(starting)
    try:
        headers = {'AccountKey':'XaA8uKDzTjevY+gR1sjz0A=='}
        results = requests.get('http://datamall2.mytransport.sg/ltaodataservice/CarParkAvailabilityv2?$skip=' + str(starting),headers=headers).text
        results = json.loads(results)
        if len(results['value']) == 0:
            break
        parking.extend(results['value'])
        starting += 500
    except: break
        
pd.DataFrame(parking).to_clipboard(index=False)

In [166]:
lotses = []
for lat1, long1 in malls[["Latitude","Longitude"]].values.tolist():
    lots = []
    developments = []
    for location, development, available in pd.DataFrame(parking)[["Location","Development","AvailableLots"]].values.tolist():
        try:
            lat2, long2 = location.split()
            distance = dist(lat1, long1, float(lat2), float(long2))
            if distance < 1:
                lots.append(available)
                developments.append(development + "(" + str(available) + ")")
        except:
            pass
    
    lotses.append([sum(lots), ", ".join(developments)])
    
pd.DataFrame(lotses).to_clipboard(index=False,header=False)

# Volume

In [197]:
## BUS

passengers = []
starting = 0
while True:
#     print(starting)
    try:
        headers = {'AccountKey':'XaA8uKDzTjevY+gR1sjz0A=='}
#         results = requests.get('http://datamall2.mytransport.sg/ltaodataservice/PV/ODBus?$skip=' + str(starting),headers=headers).text
        results = requests.get('http://datamall2.mytransport.sg/ltaodataservice/PV/Bus?$skip=' + str(starting),headers=headers).text
        results = json.loads(results)
        if len(results['value']) == 0:
            break
        passengers.extend(results['value'])
        starting += 500
    except: break

In [7]:
passengers = pd.read_csv("Documents/Geospatial/GEOSPATIAL/origin_destination_bus_202103.csv")

destination = passengers.groupby("ORIGIN_PT_CODE").pipe(lambda x: x.TOTAL_TRIPS.sum())
origin = passengers.groupby("DESTINATION_PT_CODE").pipe(lambda x: x.TOTAL_TRIPS.sum())
passengers = pd.DataFrame([destination, origin]).T
passengers = passengers.reset_index()
passengers.columns = ["BusStopCode","Origin","Destination"]

passengers = pd.read_csv("Documents/Geospatial/GEOSPATIAL/transport_node_bus_202103.csv")

tapIn = passengers.groupby("PT_CODE").pipe(lambda x: x.TOTAL_TAP_IN_VOLUME.sum())
tapOut = passengers.groupby("PT_CODE").pipe(lambda x: x.TOTAL_TAP_OUT_VOLUME.sum())
passengers = pd.DataFrame([tapIn, tapOut]).T
passengers = passengers.reset_index()
passengers.columns = ["BusStopCode","In","Out"]

# bus.drop(["Origin","Destination"],axis=1).merge(passengers).to_clipboard(index=False)

In [8]:
passengers

,BusStopCode,In,Out
0,1012,26082,25850
1,1013,17343,8757
2,1019,11815,22550
3,1029,27486,20168
4,1039,43087,41931
...,...,...,...
5011,99139,34,21456
5012,99161,88,720
5013,99171,666,3448
5014,99181,12069,13049


In [6]:
## TRAIN

passengerst = []
starting = 0
while True:
#     print(starting)
    try:
        headers = {'AccountKey':'XaA8uKDzTjevY+gR1sjz0A=='}
        results = requests.get('http://datamall2.mytransport.sg/ltaodataservice/PV/Train?$skip=' + str(starting),headers=headers).text
        results = json.loads(results)
        if len(results['value']) == 0:
            break
        passengerst.extend(results['value'])
        starting += 500
    except: break

In [7]:
results

{'fault': {'faultstring': 'Rate limit quota violation. Quota limit  exceeded. Identifier : XaA8uKDzTjevY+gR1sjz0A==',
  'detail': {'errorcode': 'policies.ratelimit.QuotaViolation'}}}

In [2]:
trainRaw = gpd.read_file("Documents/Geospatial/GEOSPATIAL/TrainStation_Jan2020/MRTLRTStnPtt.shp")
trainRaw["geometry"] = trainRaw["geometry"].to_crs(epsg=4326)
trainStationExits = gpd.read_file("Documents/Geospatial/GEOSPATIAL/TrainStationExit_Jan2020/TrainStationExit06032020.shp")
train = trainRaw.merge(trainStationExits, on="STN_NAME", how="right")[["STN_NAME","geometry_x","geometry_y"]]
train.columns = ["Name","Station","Exit"]
train["Exit"] = train["Exit"].apply(shapely.wkt.dumps).str.replace(" Z","").str.replace(" 0\.0+","").apply(shapely.wkt.loads)
train = train.melt(id_vars="Name",var_name="Type",value_name="geometry")
train = gpd.GeoDataFrame(train,geometry="geometry")
train["Longitude"] = train["geometry"].x
train["Latitude"] = train["geometry"].y
train["Name"] = train["Name"].str.replace(" .RT STATION","")

In [3]:
train

,Name,Type,geometry,Longitude,Latitude
0,CANBERRA,Station,POINT (103.82971 1.44314),103.829712,1.443137
1,CANBERRA,Station,POINT (103.82971 1.44314),103.829712,1.443137
2,CANBERRA,Station,POINT (103.82971 1.44314),103.829712,1.443137
3,CANBERRA,Station,POINT (103.82971 1.44314),103.829712,1.443137
4,CANBERRA,Station,POINT (103.82971 1.44314),103.829712,1.443137
...,...,...,...,...,...
1183,SENTOSA,Exit,POINT (103.82251 1.26509),103.822510,1.265085
1184,TUAS CRESCENT,Exit,POINT (103.64896 1.32076),103.648959,1.320764
1185,TUAS CRESCENT STATION,Exit,POINT (103.64929 1.32124),103.649287,1.321244
1186,TUAS WEST,Exit,POINT (103.63928 1.32987),103.639284,1.329872


In [4]:
passengerst = pd.read_csv("Documents/Geospatial/GEOSPATIAL/origin_destination_train_202103.csv")

destination = passengerst.groupby("ORIGIN_PT_CODE").pipe(lambda x: x.TOTAL_TRIPS.sum())
origin = passengerst.groupby("DESTINATION_PT_CODE").pipe(lambda x: x.TOTAL_TRIPS.sum())
passengerst = pd.DataFrame([destination, origin]).T
passengerst = passengerst.reset_index()
passengerst.columns = ["BusStopCode","Origin","Destination"]

train.merge(passengerst)[["Origin","Destination"]].to_clipboard()

MergeError: No common columns to perform merge on. Merge options: left_on=None, right_on=None, left_index=False, right_index=False

In [6]:
train

,BusStopCode,Origin,Destination
0,BP10,87603,94728
1,BP11,93150,75604
2,BP12,73068,54561
3,BP13,40305,25649
4,BP2,85711,66364
...,...,...,...
154,SW6,88303,83495
155,SW7,66288,60525
156,SW8,50989,50352
157,TE1,27527,27556


In [171]:
bus

,ServiceNo,Operator,Direction,StopSequence,BusStopCode,Distance,WD_FirstBus,WD_LastBus,SAT_FirstBus,SAT_LastBus,SUN_FirstBus,SUN_LastBus,RoadName,Description,Latitude,Longitude,Alt,Origin,Destination
0,123,SBST,1,43,14539,21.3,701,56,654,100,709,51,Beach View,Beach Station Bus Terminal,1.251427,103.818545,11.108041,3912.0,3163.0
1,123,SBST,2,1,14539,0.0,600,2345,600,2345,615,2345,Beach View,Beach Station Bus Terminal,1.251427,103.818545,11.108041,3912.0,3163.0
2,123,SBST,2,2,14529,0.8,603,2347,603,2348,617,2348,Artillery Ave,Opp Village Hotel,1.253226,103.821217,18.755095,1209.0,8.0
3,123,SBST,1,41,14519,18.9,649,46,644,50,701,43,Sentosa Gateway,Resorts World Sentosa,1.253522,103.825703,8.179315,372.0,11023.0
4,188R,SMRT,1,28,14519,23.4,-,-,817,2314,817,2314,Sentosa Gateway,Resorts World Sentosa,1.253522,103.825703,8.179315,372.0,11023.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26251,981,SMRT,2,3,47141,0.6,1702,1932,1202,1402,-,-,Senoko Rd,Bef Sc Auto,1.467570,103.803678,3.306511,NaN,NaN
26252,170,SBST,1,68,46239,34.2,614,150,604,147,610,151,Jln Datin Halimah,Larkin Ter,1.493904,103.744033,9.234995,NaN,NaN
26253,170,SBST,2,1,46239,0.0,520,30,530,30,530,30,Jln Datin Halimah,Larkin Ter,1.493904,103.744033,9.234995,NaN,NaN
26254,NR8,SMRT,1,60,43579,24.1,-,-,43,308,-,-,NaN,NaN,NaN,NaN,10.000000,NaN,NaN


In [196]:
passengers.sort_values("Origin", ascending=False)

,BusStopCode,Origin,Destination
975,22009,1602978.0,1383678.0
2257,46009,1225466.0,942825.0
4087,75009,1099629.0,1092258.0
2643,52009,827177.0,770928.0
3145,59009,826970.0,534553.0
...,...,...,...
1669,31201,NaN,8.0
2169,44989,NaN,128.0
3258,59609,NaN,79.0
3506,64259,NaN,40.0
